In [1]:
!pip install kaggle

In [6]:
!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [7]:
!kaggle competitions download -c test-recsys

 91% 137M/150M [00:01<00:00, 110MB/s]
100% 150M/150M [00:01<00:00, 88.9MB/s]
 90% 129M/144M [00:02<00:00, 55.9MB/s]
100% 144M/144M [00:02<00:00, 68.4MB/s]
 87% 135M/156M [00:01<00:00, 119MB/s]
100% 156M/156M [00:01<00:00, 143MB/s]
 82% 89.0M/108M [00:01<00:00, 42.9MB/s]
100% 108M/108M [00:01<00:00, 67.5MB/s] 
 99% 95.0M/96.2M [00:00<00:00, 132MB/s]
100% 96.2M/96.2M [00:00<00:00, 148MB/s]
 92% 126M/137M [00:03<00:00, 23.9MB/s]
100% 137M/137M [00:03<00:00, 40.3MB/s]
 98% 58.0M/59.0M [00:00<00:00, 55.4MB/s]
100% 59.0M/59.0M [00:00<00:00, 80.5MB/s]
 85% 41.0M/48.0M [00:00<00:00, 56.6MB/s]
100% 48.0M/48.0M [00:00<00:00, 89.1MB/s]
  0% 0.00/11.2M [00:00<?, ?B/s]
100% 11.2M/11.2M [00:00<00:00, 103MB/s]
 90% 67.0M/74.7M [00:00<00:00, 45.2MB/s]
100% 74.7M/74.7M [00:00<00:00, 127MB/s] 
 56% 9.00M/16.1M [00:00<00:00, 40.2MB/s]
100% 16.1M/16.1M [00:00<00:00, 53.8MB/s]
 83% 24.0M/29.0M [00:00<00:00, 45.5MB/s]
100% 29.0M/29.0M [00:00<00:00, 74.3MB/s]
 27% 5.00M/18.8M [00:00<00:00, 17.7MB/s]
100% 18.

In [8]:
import os
import pandas as pd
import zipfile
from tqdm import tqdm

In [9]:
df_list = []
products_files = [filename for filename in os.listdir() if filename.startswith('tab_2')]
products_files.sort()

for zipname in tqdm(products_files[-5:]):
  zf = zipfile.ZipFile(zipname)
  for name in tqdm(zf.namelist()):
    df_list.append(pd.read_csv(zf.open(name)))

100%|██████████| 5/5 [00:38<00:00,  7.68s/it]


In [10]:
products_data = pd.concat(df_list)

In [11]:
products_data.head()

,user_id,order_id,line_item_id,price,quantity,discount,product_name,product_id,brand_name,master_category_id,parent_category_id
0,51,13911065,145635438,149.899994,1,0.0,Кукуруза сладкая в вакууме 230 г,6005183,Без бренда,85.0,84
1,51,13911065,145635468,74.900002,1,0.0,Морковь Белая Дача сочная 250 г,15148,Белая Дача,85.0,84
2,51,13911065,145635515,149.899994,5,0.0,Томаты Черри Мое лето Санрайз красные 250 г,63057,Мое лето,85.0,84
3,51,13911065,145635542,99.900002,1,0.0,Огурцы Луховицкие,3497570,Без бренда,85.0,84
4,51,13911065,145635579,59.900002,2,0.0,Томаты черри сливка,688,NaN,85.0,84


In [12]:
zf = zipfile.ZipFile('/content/sample_submission.csv.zip')
sample_submission = pd.read_csv(zf.open('sample_submission.csv'))

In [13]:
K = 50
new_sub = sample_submission.copy()
for row in tqdm(sample_submission.iterrows(), total=sample_submission.shape[0]):
  user_id = row[1]['Id']
  topK_list = products_data[products_data['user_id'] == user_id].groupby('product_id')['quantity'].sum().sort_values(ascending=False)[:K].index.tolist()
  topK_list = [str(i) for i in topK_list]
  topK_str = ' '.join(topK_list)
  new_sub.loc[new_sub['Id'] == user_id, ['Predicted']] = topK_str

100%|██████████| 107068/107068 [48:54<00:00, 36.49it/s]


In [19]:
new_sub.to_csv('space_inside.csv', index=False)